In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import nfl_data_py as nfl
import numpy as np

In [2]:
def redzone_touch_share(gsis_it_id: str, year: int) -> pd.DataFrame:

    import nfl_data_py as nfl
    import pandas as pd
    
    df_plays = nfl.import_pbp_data([year])
    weekly = nfl.import_weekly_data([year])
    roster = nfl.import_weekly_rosters([year])
    
    
    player_info = roster[roster['gsis_it_id'] == gsis_it_id]
    if player_info.empty:
        print(f"No se encontró el jugador con gsis_it_id = {gsis_it_id} en el roster de {year}")
        return None

    player_id = player_info.iloc[0]['player_id']

   
    redzone_plays = df_plays[df_plays['yardline_100'].notna() & (df_plays['yardline_100'] <= 20)]


    df_player = redzone_plays[
        (redzone_plays['rusher_player_id'] == player_id) | 
        (redzone_plays['receiver_player_id'] == player_id)
    ].copy()

    if df_player.empty:
        print(f"El jugador {gsis_it_id} no tuvo jugadas en la zona roja en {year}.")
        return pd.DataFrame()

   
    df_player['touch_type'] = df_player.apply(
        lambda row: 'rush' if row['rusher_player_id'] == player_id else 'reception',
        axis=1
    )

   
    df_player['touchdown'] = df_player['touchdown'].astype(int)

   
    summary = df_player.groupby(['posteam', 'defteam', 'week']).agg(
        redzone_touches=('touch_type', 'count'),
        touchdowns=('touchdown', 'sum'),
        total_yards=('yards_gained', 'sum')
    ).reset_index()

    return summary.sort_values(by='week')